In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from time import sleep
import random
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

import zipfile
import shutil

import requests
import pandas as pd

import json

from bs4 import BeautifulSoup

# ---------------------------------------
# Path
import sys
sys.path.append('../')

import warnings
#warnings.filterwarnings('ignore')

from src.support import get_dotd, get_df_dotd

### Driver of the day

In [2]:
try:
    df_dotd = pd.read_csv('../data/dotd.csv')
except FileNotFoundError:
    soup = get_dotd()
    df_dotd = get_df_dotd(soup)
    df_dotd.to_csv('../data/dotd.csv', index = False)

### Circuits

In [3]:
url = 'http://ergast.com/api/f1/2023/circuits.json'

response = requests.get(url)

print(response.status_code)

cont = response.json()

200


In [4]:
circuits = cont['MRData']['CircuitTable']['Circuits']

In [5]:
df_circ = pd.DataFrame(circuits)

In [6]:
df_circ

,circuitId,url,circuitName,Location
0,albert_park,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Albert Park Grand Prix Circuit,"{'lat': '-37.8497', 'long': '144.968', 'locali..."
1,americas,http://en.wikipedia.org/wiki/Circuit_of_the_Am...,Circuit of the Americas,"{'lat': '30.1328', 'long': '-97.6411', 'locali..."
2,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,"{'lat': '26.0325', 'long': '50.5106', 'localit..."
3,baku,http://en.wikipedia.org/wiki/Baku_City_Circuit,Baku City Circuit,"{'lat': '40.3725', 'long': '49.8533', 'localit..."
4,catalunya,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,Circuit de Barcelona-Catalunya,"{'lat': '41.57', 'long': '2.26111', 'locality'..."
5,hungaroring,http://en.wikipedia.org/wiki/Hungaroring,Hungaroring,"{'lat': '47.5789', 'long': '19.2486', 'localit..."
6,interlagos,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Jo...,Autódromo José Carlos Pace,"{'lat': '-23.7036', 'long': '-46.6997', 'local..."
7,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,Jeddah Corniche Circuit,"{'lat': '21.6319', 'long': '39.1044', 'localit..."
8,losail,http://en.wikipedia.org/wiki/Losail_Internatio...,Losail International Circuit,"{'lat': '25.49', 'long': '51.4542', 'locality'..."
9,marina_bay,http://en.wikipedia.org/wiki/Marina_Bay_Street...,Marina Bay Street Circuit,"{'lat': '1.2914', 'long': '103.864', 'locality..."


In [7]:
# URL (no api)
url_circ = 'https://en.wikipedia.org/wiki/Circuit_of_the_Americas'
#url_circ = 'https://en.wikipedia.org/wiki/Albert_Park_Circuit'

res_circ = requests.get(url_circ)
# Comprobamos que obtenemos un 200
print(res_circ.status_code)

soup = BeautifulSoup(res_circ.content, 'html.parser')

200


In [23]:
def get_add_circuit_info(url):

    print(url)

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

    else:
        pass

    try:
        table = soup.find("table", {"class": "infobox vcard"})
        rows = table.findAll("tr")

        capacity = 'NA'
        website = 'NA'
        architect = 'NA'

        for row in rows:
            header_cell = row.find("th")
            if header_cell:

                text = header_cell.text.strip().lower()

                if text == 'capacity':
                    capacity = row.find("td").text

                elif text == 'website':
                    website = row.find("td").text

                elif text == 'architect':
                    architect = row.find("td").text

        tup = (capacity, website, architect)
        print(tup)
        return tup
    
    except:

        try:
            url = soup.find('a', string=re.compile('Circuit', re.IGNORECASE)).get("href")

            root = 'https://en.wikipedia.org'

            if root not in url:

                url = root + url

            return get_add_circuit_info(url)

        except:
            print('Error')
           


In [24]:
a = get_add_circuit_info('https://en.wikipedia.org/wiki/Las_Vegas_Grand_Prix#Circuit')

https://en.wikipedia.org/wiki/Las_Vegas_Grand_Prix#Circuit
https://en.wikipedia.org/wiki/Las_Vegas_Strip_Circuit
('100,000', 'NA', 'Carsten Tilke')


In [25]:
print(a)

('100,000', 'NA', 'Carsten Tilke')


In [106]:
for link in links:

    print(link.get('href'))

/wiki/Las_Vegas_Strip_Circuit
/wiki/Las_Vegas_Strip_Circuit
/wiki/Las_Vegas_Strip_Circuit
/wiki/Las_Vegas_Strip_Circuit
https://www.fia.com/sites/default/files/decision-document/2023%20Las%20Vegas%20Grand%20Prix%20-%20Event%20Notes%20-%20Circuit%20Map%20V3.pdf


In [80]:
df_circ['url'].apply(get_add_circuit_info)

http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit
http://en.wikipedia.org/wiki/Circuit_of_the_Americas
http://en.wikipedia.org/wiki/Bahrain_International_Circuit
http://en.wikipedia.org/wiki/Baku_City_Circuit
http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya
http://en.wikipedia.org/wiki/Hungaroring
http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Jos%C3%A9_Carlos_Pace
http://en.wikipedia.org/wiki/Jeddah_Street_Circuit
http://en.wikipedia.org/wiki/Losail_International_Circuit
http://en.wikipedia.org/wiki/Marina_Bay_Street_Circuit
http://en.wikipedia.org/wiki/Miami_International_Autodrome
http://en.wikipedia.org/wiki/Circuit_de_Monaco
http://en.wikipedia.org/wiki/Autodromo_Nazionale_Monza
http://en.wikipedia.org/wiki/Red_Bull_Ring
http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Hermanos_Rodr%C3%ADguez
http://en.wikipedia.org/wiki/Silverstone_Circuit
http://en.wikipedia.org/wiki/Circuit_de_Spa-Francorchamps
http://en.wikipedia.org/wiki/Suzuka_Circuit
https://en.wikipedia.org/wiki

AttributeError: 'NoneType' object has no attribute 'findAll'